In [ ]:
# Colab-ready script
import os
import time
import torch
import torch.nn as nn
import torchvision.transforms as T
import psutil
import GPUtil
import numpy as np
import cv2
from google.colab import drive
from PIL import Image
from ptflops import get_model_complexity_info
from torch.utils.data import Dataset, DataLoader
from torchvision import models

# ------------------------------
# 1. Accept Google Drive links from user
# ------------------------------
suim_link = input("Enter Google Drive link for SUIM dataset: ").strip()
uieb_link = input("Enter Google Drive link for UIEB dataset: ").strip()

# Extract file ID from Google Drive link
def extract_file_id(link):
    if "id=" in link:
        return link.split("id=")[1]
    elif "/d/" in link:
        return link.split("/d/")[1].split("/")[0]
    else:
        return None

suim_id = extract_file_id(suim_link)
uieb_id = extract_file_id(uieb_link)

if suim_id:
    os.system(f"gdown --id {suim_id} -O suim.zip")
    os.system("unzip -q suim.zip -d suim_dataset")
else:
    print("Invalid SUIM dataset link!")

if uieb_id:
    os.system(f"gdown --id {uieb_id} -O uieb.zip")
    os.system("unzip -q uieb.zip -d uieb_dataset")
else:
    print("Invalid UIEB dataset link!")

# ------------------------------
# 2. Dummy Dataset Loader (Replace with actual dataset logic)
# ------------------------------
class SegmentationDataset(Dataset):
    def __init__(self, img_dir, mask_dir, transform=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.img_list = os.listdir(img_dir)
        self.transform = transform

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_list[idx])
        mask_path = os.path.join(self.mask_dir, self.img_list[idx].replace(".jpg", ".png"))

        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return image, mask

transform = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor()
])

# Example paths (Update these to match dataset structure)
suim_dataset = SegmentationDataset("suim_dataset/images", "suim_dataset/masks", transform)
uieb_dataset = SegmentationDataset("uieb_dataset/images", "uieb_dataset/masks", transform)

suim_loader = DataLoader(suim_dataset, batch_size=1, shuffle=False)
uieb_loader = DataLoader(uieb_dataset, batch_size=1, shuffle=False)

# ------------------------------
# 3. Dummy Models (Replace with actual)
# ------------------------------
class DummySegFormer(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = models.resnet18(pretrained=False)
    def forward(self, x):
        return torch.sigmoid(torch.randn(x.size(0), 1, x.size(2), x.size(3)))

# Example models
models_dict = {
    "Fast-EUVP-UW-SegFormer-B0 (Proposed)": DummySegFormer(),
    "UW-SegFormer-B0 (No enhancement)": DummySegFormer(),
    "ResNet-50 DeepLab v3+ (FP16)": models.segmentation.deeplabv3_resnet50(pretrained=False)
}

# ------------------------------
# 4. Metrics Calculation Functions
# ------------------------------
def compute_miou(pred, target):
    pred = pred > 0.5
    target = target > 0.5
    intersection = (pred & target).sum().float()
    union = (pred | target).sum().float()
    return (intersection / union).item()

def measure_latency(model, device, input_size=(1, 3, 256, 256), runs=50):
    model.to(device)
    model.eval()
    dummy_input = torch.randn(input_size).to(device)
    torch.cuda.synchronize()
    start = time.time()
    for _ in range(runs):
        _ = model(dummy_input)
    torch.cuda.synchronize()
    end = time.time()
    avg_latency = (end - start) / runs
    return avg_latency

def measure_gpu_power():
    gpus = GPUtil.getGPUs()
    if not gpus:
        return None, None
    gpu = gpus[0]
    return gpu.load * 100, gpu.powerUsage

# ------------------------------
# 5. Evaluation Loop
# ------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

results = []
for model_name, model in models_dict.items():
    print(f"\nEvaluating {model_name}...")

    # Params & GFLOPs
    with torch.cuda.device(0):
        macs, params = get_model_complexity_info(model, (3, 256, 256), as_strings=False, print_per_layer_stat=False)

    # Latency & FPS
    latency = measure_latency(model, device)
    fps = 1 / latency

    # GPU utilization & Power
    gpu_util, power = measure_gpu_power()

    # mIoU on SUIM & UIEB
    miou_suim = np.mean([compute_miou(model(img.unsqueeze(0).to(device)), mask.unsqueeze(0).to(device))
                         for img, mask in suim_loader])
    miou_uieb = np.mean([compute_miou(model(img.unsqueeze(0).to(device)), mask.unsqueeze(0).to(device))
                         for img, mask in uieb_loader])

    results.append({
        "Model": model_name,
        "Params(M)": round(params / 1e6, 2),
        "GFLOPs": round(macs / 1e9, 2),
        "Latency(s)": round(latency, 4),
        "FPS": round(fps, 2),
        "GPU Util(%)": round(gpu_util, 2) if gpu_util else None,
        "Power(W)": round(power, 2) if power else None,
        "mIoU (SUIM)": round(miou_suim, 4),
        "mIoU (UIEB)": round(miou_uieb, 4)
    })

# ------------------------------
# 6. Print Results
# ------------------------------
import pandas as pd
df = pd.DataFrame(results)
print("\nFinal Evaluation Results:")
print(df)
